# Sentiment Classification on Text Data 

**Authors:** <br>
Ian Colson (colso031) <br>
Maitrayee Deka (deka0031)<br>
Sneha Patri(patri316) <br>
Aditi Patil (patil112) <br>
Benjamin Swenson(swen0754) <br>

---